# <center>__MÉTODOS NUMÉRICOS__</center>
## <center>__PROJETO DA UNIDADE 2__</center>
#### <center>__Equipe: João Marcos e Wesley Wilson__</center>

<div class="alert alert-block alert-info">
1. INTRODUÇÃO
</div>

Sistemas de recomendação tornaram-se fundamentais para ajudar usuários a descobrirem novos conteúdos em meio à vasta oferta digital. No contexto dos jogos, essas ferramentas podem sugerir títulos alinhados aos gostos individuais, aumentando o engajamento e a satisfação do jogador.

Nossa abordagem se baseia em duas técnicas principais de álgebra linear e processamento de dados. A primeira é a **filtragem colaborativa** através da **Decomposição de Valores Singulares (SVD)**, um método de fatoração matricial que nos permite encontrar padrões latentes nas avaliações dos usuários. A segunda é a **filtragem baseada em conteúdo**, onde usamos a **similaridade de cossenos** para recomendar jogos com base em suas características, como gênero e nome. Por fim, combinamos essas abordagens em um sistema híbrido para fornecer recomendações mais robustas e solucionar desafios como o problema de *cold start*.


<div class="alert alert-block alert-info">
2. DESCRIÇÃO DO PROBLEMA
</div>

O objetivo central deste projeto é desenvolver um sistema de recomendação de jogos para a plataforma PC. O sistema deve ser capaz de gerar sugestões personalizadas com base no histórico de avaliações de cada usuário, bem como encontrar jogos similares a um título específico.

Para isso, enfrentamos desafios inerentes a sistemas de recomendação:
- **Esparsidade da Matriz:** A maioria dos usuários avalia apenas uma pequena fração dos jogos disponíveis, resultando em uma matriz de utilidade (usuário-jogo) extremamente esparsa.
- **Cold Start:** O sistema precisa lidar com novos usuários (que ainda não avaliaram nada) ou novos jogos (que ainda não receberam avaliações).
- **Escalabilidade:** A solução deve ser computacionalmente viável, mesmo com um grande número de usuários e jogos.

<div class="alert alert-block alert-info">
3. MÉTODOS APLICADOS À SOLUÇÃO
</div>

Para resolver o problema, empregamos uma abordagem híbrida que combina os pontos fortes de diferentes métodos numéricos:

1.  **Filtragem Colaborativa com SVD:** Este método se baseia na ideia de que usuários com gostos similares no passado tenderão a ter gostos similares no futuro. Construímos uma matriz de utilidade $A$, onde as linhas representam usuários, as colunas representam jogos e os valores são as avaliações. Como essa matriz é esparsa, usamos a **Decomposição de Valores Singulares (SVD)** para decompô-la em três matrizes: $A = U \Sigma V^T$. A SVD nos ajuda a encontrar fatores latentes (conceitos abstratos como "jogos de mundo aberto com boa narrativa" ou "RPGs de ação intensa") e a preencher as avaliações ausentes na matriz original, permitindo-nos prever a nota que um usuário daria a um jogo que ele ainda não jogou.

2.  **Filtragem Baseada em Conteúdo:** Este método recomenda itens com base em suas características. Criamos um perfil para cada jogo, combinando seu nome e gêneros em uma "sopa de características". Usamos a técnica **TF-IDF (Term Frequency-Inverse Document Frequency)** para vetorizar esse texto, convertendo as características em um espaço vetorial. Em seguida, calculamos a **similaridade de cossenos** entre os vetores de todos os jogos. O resultado é uma matriz de similaridade que nos diz o quão parecido um jogo é do outro, permitindo-nos recomendar títulos similares a um que o usuário já gostou. Este método é especialmente útil para resolver o problema de *cold start* para novos jogos.

3.  **Estratégia de Popularidade (para *Cold Start* de Usuário):** Para novos usuários sem histórico de avaliações, os métodos acima não funcionam. Nesse caso, adotamos uma estratégia simples e eficaz: recomendar os jogos mais populares e bem avaliados, combinando dados da IGDB e da Steam. Isso serve como um ponto de partida seguro para que o novo usuário comece a interagir com o sistema.


<div class="alert alert-block alert-info">
4. IMPLEMENTAÇÃO
</div>

#### 4.1. Configuração Inicial e Coleta de Dados

In [ ]:
import requests
import pandas as pd
import numpy as np
from scipy.sparse import csc_matrix
from scipy.sparse.linalg import svds
from sklearn.metrics import mean_squared_error
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

CLIENT_ID = 'Insira o Client ID aqui'
CLIENT_SECRET = 'Insira o Client Secret aqui'

In [ ]:
try:
    auth_url = 'https://id.twitch.tv/oauth2/token'
    payload = {'client_id': CLIENT_ID, 'client_secret': CLIENT_SECRET, 'grant_type': 'client_credentials'}
    resp = requests.post(auth_url, data=payload)
    resp.raise_for_status() # Lança um erro para status HTTP ruins (4xx ou 5xx)
    token = resp.json().get('access_token')
    if not token:
        raise ValueError("Token de acesso não foi obtido. Verifique as variáveis CLIENT_ID e CLIENT_SECRET.")
    headers_igdb = {'Client-ID': CLIENT_ID, 'Authorization': f'Bearer {token}'}
    print("Autenticação com a API da IGDB bem-sucedida.")
except (requests.exceptions.RequestException, ValueError) as e:
    print(f"Erro na autenticação IGDB: {e}")
    # Encerra o script se a autenticação falhar, pois os passos seguintes dependem dela
    exit()

Autenticação com a API da IGDB bem-sucedida.


In [ ]:
query = 'fields name,genres.name,rating,total_rating_count,websites.category,websites.url; where platforms = (6) & total_rating_count > 100; sort total_rating_count desc; limit 250;'
resp_games = requests.post('https://api.igdb.com/v4/games', headers=headers_igdb, data=query)
jogos_raw = resp_games.json()
print(f"{len(jogos_raw)} jogos coletados da IGDB.")

250 jogos coletados da IGDB.


Para conectar os dados da IGDB com os da Steam, precisamos extrair o ID de cada jogo na plataforma da Steam. Criamos uma função para analisar a lista de websites de cada jogo e encontrar a URL da loja Steam (categoria 13), extraindo o ID numérico dela.

In [ ]:
def get_steam_app_id(websites):
    """
    Extrai o steam_app_id de uma lista de websites retornada pela API do IGDB.
    """
    if not isinstance(websites, list):
        return None
    for site in websites:
        # Categoria 13 é a da Steam
        if site.get('category') == 13:
            url = site.get('url', '')
            parts = url.split('/app/')
            if len(parts) > 1:
                app_id_str = parts[1].split('/')[0]
                if app_id_str.isdigit():
                    return app_id_str
    return None

In [ ]:
df_igdb = pd.DataFrame(jogos_raw)
df_igdb['steam_app_id'] = df_igdb['websites'].apply(get_steam_app_id)
df_igdb = df_igdb.dropna(subset=['steam_app_id'])
df_igdb_clean = df_igdb[['id', 'name', 'rating', 'genres', 'steam_app_id']].copy()
df_igdb_clean.columns = ['igdb_id', 'name', 'avg_igdb_rating', 'genres', 'steam_app_id']
print(f"{len(df_igdb_clean)} jogos restantes após filtrar por aqueles com link para a Steam.")

215 jogos restantes após filtrar por aqueles com link para a Steam.


Agora, usamos os IDs da Steam para coletar avaliações de usuários diretamente da API da Steam. Para cada jogo, buscamos até 100 avaliações recentes. Mapeamos as avaliações "Recomendado" para 1 e "Não Recomendado" para -1, um formato adequado para o modelo SVD.

In [ ]:
records = []
app_ids_to_fetch = df_igdb_clean['steam_app_id'].dropna().unique()

print(f"Coletando avaliações da Steam para {len(app_ids_to_fetch)} jogos...")
for app_id in app_ids_to_fetch:
    try:
        url = f"https://store.steampowered.com/appreviews/{app_id}"
        params = {'json': 1, 'num_per_page': 100, 'language': 'all', 'filter': 'recent'}
        r = requests.get(url, params=params)
        r.raise_for_status()
        data = r.json()
        if data.get('success') and data.get('reviews'):
            for review in data['reviews']:
                rating = 1 if review.get('voted_up', False) else -1
                records.append({
                    'user_id': review['author']['steamid'],
                    'steam_app_id': app_id,
                    'rating': rating,
                    'recommended': review.get('voted_up', False)
                })
    except requests.exceptions.RequestException as e:
        pass

if not records:
    print("Nenhuma avaliação da Steam foi coletada. Encerrando.")
    exit()

df_steam = pd.DataFrame(records)
print(f"Total de {len(df_steam)} avaliações coletadas da Steam.")

Coletando avaliações da Steam para 215 jogos...
Total de 21259 avaliações coletadas da Steam.


#### 4.2. Implementação da Filtragem Colaborativa (SVD)

Para preparar os dados para o SVD, precisamos converter os IDs de usuário e jogo para índices numéricos contínuos (de 0 a N-1), que servirão como coordenadas na nossa matriz de utilidade.

In [ ]:
# Mapear IDs de usuário e de jogo para índices de matriz
user_ids = df_steam['user_id'].unique()
app_ids = df_steam['steam_app_id'].unique()
user_map = {uid: i for i, uid in enumerate(user_ids)}
app_map = {aid: i for i, aid in enumerate(app_ids)}
app_inv_map = {i: aid for aid, i in app_map.items()} # Mapa inverso para encontrar o ID a partir do índice

df_steam['user_idx'] = df_steam['user_id'].map(user_map)
df_steam['app_idx'] = df_steam['steam_app_id'].map(app_map)

Dividimos nosso conjunto de dados em treino (80%) e teste (20%). É crucial que essa divisão seja feita sobre as interações (avaliações), e não sobre os usuários ou jogos, para simular um cenário real onde queremos prever avaliações desconhecidas.

In [ ]:
# Divisão treino/teste
train_indices = np.random.choice(df_steam.index, size=int(0.8 * len(df_steam)), replace=False)
test_indices = df_steam.index.difference(train_indices)
train_data = df_steam.loc[train_indices]
test_data = df_steam.loc[test_indices]

print(f"Tamanho do conjunto de treino: {len(train_data)} avaliações")
print(f"Tamanho do conjunto de teste: {len(test_data)} avaliações")

Tamanho do conjunto de treino: 17007 avaliações
Tamanho do conjunto de teste: 4252 avaliações


Agora, construímos a matriz de utilidade $A$ (aqui chamada `R_train`) a partir dos dados de treino. Usamos `csc_matrix` do SciPy, que é um formato eficiente para matrizes esparsas, economizando memória.

In [ ]:
R_train = csc_matrix((train_data['rating'], (train_data['user_idx'], train_data['app_idx'])),
                     shape=(len(user_ids), len(app_ids)),
                     dtype=np.float64)

Este é o núcleo da filtragem colaborativa. Aplicamos a Decomposição de Valores Singulares (`svds` do SciPy, otimizada para matrizes esparsas) na nossa matriz de treino. Escolhemos `k=50` fatores latentes, um hiperparâmetro que representa a complexidade do nosso modelo. O resultado são as três matrizes: $U$ (fatores latentes dos usuários), $\Sigma$ (a importância de cada fator) e $V^T$ (fatores latentes dos itens). Ao multiplicá-las (`U @ sigma_diag @ Vt`), obtemos a matriz `R_est`, que é a nossa matriz de utilidade reconstruída e preenchida com as predições de rating.

In [ ]:
# Aplicar SVD
k = 50
U, sigma, Vt = svds(R_train, k=k)
sigma_diag = np.diag(sigma)

# Reconstruir a matriz de predições
R_est = U @ sigma_diag @ Vt
print(f"Matriz de predições reconstruída com shape: {R_est.shape}")

Matriz de predições reconstruída com shape: (19976, 215)


Para avaliar a qualidade do nosso modelo SVD, calculamos o **RMSE (Root Mean Squared Error)**. Essa métrica mede a diferença média entre os ratings preditos pelo modelo (`R_est`) e os ratings reais que separamos no conjunto de teste. Um RMSE menor indica um modelo mais preciso.

In [ ]:
def get_predictions(R_pred, data):
    y_true, y_pred = [], []
    for _, row in data.iterrows():
        user_idx, app_idx = row['user_idx'], row['app_idx']
        if user_idx < R_pred.shape[0] and app_idx < R_pred.shape[1]:
            y_true.append(row['rating'])
            y_pred.append(R_pred[user_idx, app_idx])
    return y_true, y_pred

y_true, y_pred = get_predictions(R_est, test_data)
if y_true:
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    print(f"RMSE do modelo SVD: {rmse:.4f}")
else:
    print("Não foi possível calcular o RMSE (conjunto de teste vazio).")

RMSE do modelo SVD: 0.9949


#### 4.3. Implementação da Filtragem Baseada em Conteúdo

Agora, mudamos para a abordagem baseada em conteúdo. O primeiro passo é criar um DataFrame unificado (`df_final`) que contenha todas as informações relevantes dos jogos (metadados da IGDB e métricas da Steam).

In [ ]:
# Calcular métricas da Steam (taxa de aprovação e contagem de reviews)
steam_metrics = df_steam.groupby('steam_app_id').agg(
    approval_rate=('recommended', 'mean'),
    review_count=('recommended', 'size')
).reset_index()

# Juntar os dados da IGDB com as métricas da Steam
df_final = df_igdb_clean.merge(steam_metrics, on='steam_app_id', how='left')
df_final['approval_rate'] = df_final['approval_rate'].fillna(0)
df_final['review_count'] = df_final['review_count'].fillna(0)

In [ ]:
# Lidar com valores 'NaN' ou ausentes na coluna de gêneros
df_final['genres'] = df_final['genres'].apply(lambda x: x if isinstance(x, list) else [])

# Função para extrair e limpar os nomes dos gêneros
def extrair_nomes_generos(lista_generos):
    if not lista_generos:
        return ""
    return " ".join([gen['name'].replace(" ", "") for gen in lista_generos])

# Criar a coluna de gêneros limpa e a "sopa de características"
df_final['genres_clean'] = df_final['genres'].apply(extrair_nomes_generos)
df_final['sopa_features'] = df_final['name'].str.lower() + " " + df_final['genres_clean'].str.lower()

In [ ]:
# Vetorização e Cálculo de Similaridade
tfidf = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf.fit_transform(df_final['sopa_features'])
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

# Mapeamento de nomes de jogos para seus índices no DataFrame
indices_map = pd.Series(df_final.index, index=df_final['name']).drop_duplicates()
print("Matriz de similaridade de cossenos construída com sucesso.")

Matriz de similaridade de cossenos construída com sucesso.


#### 4.4. Função de Recomendação Unificada

Finalmente, para amarrar todas as partes do projeto, criamos uma função "mestre" chamada `obter_recomendacoes`. Esta função é a interface final do nosso sistema e decide qual estratégia de recomendação usar com base nos parâmetros fornecidos:

1.  **Recomendação por Conteúdo:** Se um `titulo_jogo` é fornecido, a função usa a matriz `cosine_sim` para encontrar os jogos mais similares.
2.  **Recomendação Personalizada (SVD):** Se um `user_id` conhecido é fornecido, a função usa a matriz de predições `R_est` para encontrar os jogos que o modelo SVD prevê que aquele usuário mais gostaria (e que ele ainda não avaliou).
3.  **Recomendação por Popularidade (*Cold Start*):** Se nenhum parâmetro (ou um `user_id` desconhecido) é fornecido, a função recorre a uma lista de jogos de "alto consenso", que são os mais bem avaliados tanto na IGDB quanto na Steam. Esta é a nossa solução para o problema de *cold start* de novos usuários.


In [ ]:
def obter_recomendacoes(user_id=None, titulo_jogo=None, num_recs=5):
    """
    Função mestre de recomendação.
    - Se 'titulo_jogo' é fornecido, retorna recomendações baseadas em conteúdo.
    - Se 'user_id' é fornecido, retorna recomendações personalizadas (SVD).
    - Se nenhum é fornecido, retorna recomendações de popularidade (cold start).
    """
    # --- Estratégia 1: Recomendação por Conteúdo ---
    if titulo_jogo:
        if titulo_jogo not in indices_map:
            print(f"\nErro: O jogo '{titulo_jogo}' não foi encontrado em nossa base de dados.")
            return

        print(f"\n--- Recomendações baseadas em similaridade com '{titulo_jogo}' ---")
        idx = indices_map[titulo_jogo]
        sim_scores = list(enumerate(cosine_sim[idx]))
        sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
        sim_scores = sim_scores[1:num_recs+1]
        game_indices = [i[0] for i in sim_scores]
        recomendacoes = df_final['name'].iloc[game_indices]
        for i, rec in enumerate(recomendacoes):
            print(f"{i+1}. {rec}")
        return

    # --- Estratégia 2: Recomendação Personalizada (SVD) ---
    if user_id and user_id in user_map:
        print(f"\n--- Recomendações personalizadas para o usuário {user_id} ---")
        user_idx = user_map[user_id]
        user_ratings = R_est[user_idx, :]

        # Ordenar os jogos pela predição de rating
        sorted_indices = np.argsort(user_ratings)[::-1]

        # Filtrar jogos que o usuário já avaliou
        jogos_avaliados_idx = train_data[train_data['user_id'] == user_id]['app_idx'].values
        recomendacoes_idx = [idx for idx in sorted_indices if idx not in jogos_avaliados_idx]

        # Pegar os top N jogos e converter de volta para nome
        count = 0
        for idx in recomendacoes_idx:
            if count >= num_recs:
                break
            steam_app_id = app_inv_map.get(idx)
            if steam_app_id:
                nome_jogo = df_final[df_final['steam_app_id'] == steam_app_id]['name'].values
                if len(nome_jogo) > 0:
                    print(f"{count+1}. {nome_jogo[0]}")
                    count += 1
        return

    # --- Estratégia 3: Recomendação por Popularidade (Cold Start) ---
    print("\n--- Recomendações para novos usuários (jogos mais populares) ---")
    top_hybrid = df_final[
        (df_final['avg_igdb_rating'] > 85) &
        (df_final['approval_rate'] > 0.95) &
        (df_final['review_count'] > 50)
    ].sort_values(by=['approval_rate', 'avg_igdb_rating'], ascending=False)

    for i, row in top_hybrid.head(num_recs).iterrows():
        print(f"{i+1}. {row['name']} (Nota IGDB: {row['avg_igdb_rating']:.1f}, Aprovação Steam: {row['approval_rate']:.0%})")
    return

#### 4.5. Exemplos de Uso

In [ ]:
# Exemplo 1: Recomendação para um novo usuário (ou usuário desconhecido)
obter_recomendacoes()


--- Recomendações para novos usuários (jogos mais populares) ---
3. Portal 2 (Nota IGDB: 91.6, Aprovação Steam: 100%)
131. Death Stranding (Nota IGDB: 86.3, Aprovação Steam: 99%)
7. Half-Life 2 (Nota IGDB: 90.5, Aprovação Steam: 98%)
62. Resident Evil 2 (Nota IGDB: 87.6, Aprovação Steam: 98%)
33. Undertale (Nota IGDB: 86.2, Aprovação Steam: 98%)


In [ ]:
# Exemplo 2: Encontrar jogos similares a 'The Witcher 3: Wild Hunt'
# Nota: O jogo precisa estar na nossa base de dados para funcionar.
if 'The Witcher 3: Wild Hunt' in indices_map:
    obter_recomendacoes(titulo_jogo='The Witcher 3: Wild Hunt', num_recs=3)
else:
    print("\n'The Witcher 3: Wild Hunt' não foi encontrado na base de dados coletada para este exemplo.")


--- Recomendações baseadas em similaridade com 'The Witcher 3: Wild Hunt' ---
1. The Witcher 3: Wild Hunt - Hearts of Stone
2. The Witcher
3. The Witcher 2: Assassins of Kings


In [ ]:
# Exemplo 3: Recomendação personalizada para um usuário existente
# Pegamos um usuário aleatório do nosso conjunto de dados para simular
if 'df_steam' in locals() and not df_steam.empty:
    random_user_id = df_steam['user_id'].sample(1).iloc[0]
    obter_recomendacoes(user_id=random_user_id, num_recs=5)


--- Recomendações personalizadas para o usuário 76561198333471170 ---
1. Little Nightmares
2. Assassin's Creed III
3. Half-Life
4. BioShock 2
5. Dragon Age II
